In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

# --------------------------
# 1. 全局配置
# --------------------------
RENT_BASE_URL = "https://zhangjiakou.zu.fang.com/house-a014963/"  
TARGET_PAGES = 5  
WAIT_TIME = 15  
RETRY_DELAY = 5  
MAX_RETRIES = 3  


# --------------------------
# 2. 初始化Chrome浏览器
# --------------------------
def init_browser():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.maximize_window()
    return driver


# --------------------------
# 3. 提取单条租房信息（面积和月租金）
# --------------------------
def extract_rent_info(house_dl):
    """
    提取单条租房核心信息：房屋面积、月租金
    参考元素定位逻辑：
    - 月租金：通过相关价格区域提取
    - 房屋面积：p.font15 mt12 bold的文本分割后获取
    """
    rent_info = {"面积": "", "月租金": ""}
    
    try:
        # 提取房屋面积
        basic_p = house_dl.find("p", class_="font15 mt12 bold")
        if basic_p:
            basic_text = basic_p.get_text(strip=True)
            basic_segments = [seg.strip() for seg in basic_text.split("|") if seg.strip()]
            if len(basic_segments) >= 3:
                rent_info["面积"] = basic_segments[2]
        
        # 提取月租金（假设月租金在div.moreInfo区域）
        more_info_div = house_dl.find("div", class_="moreInfo")
        if more_info_div:
            price_num = more_info_div.find("span", class_="price").get_text(strip=True)
            price_unit = more_info_div.get_text(strip=True).replace(price_num, "")
            rent_info["月租金"] = f"{price_num}{price_unit}"
    
    except Exception as e:
        print(f"提取房源信息时出错：{str(e)}")
    
    return rent_info


# --------------------------
# 4. 单页爬取
# --------------------------
def crawl_single_page(driver, page):
    """
    单页爬取租房信息，带重试机制
    :param page: 当前页码
    :return: 当前页有效租房数据列表
    """
    retry_count = 0
    current_url = RENT_BASE_URL if page == 1 else f"{RENT_BASE_URL}i{30 + page}/"

    while retry_count < MAX_RETRIES:
        retry_count += 1
        try:
            driver.get(current_url)
            print(f"[第{retry_count}次重试] 正在加载租房第{page}页：{current_url}")
            
            # 等待页面核心元素加载
            WebDriverWait(driver, WAIT_TIME).until(
                EC.presence_of_element_located((By.CLASS_NAME, "list"))
            )
            time.sleep(2)  # 补充等待动态内容渲染
            
            # 解析页面HTML
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "lxml")
            # 定位所有房源<dl>元素
            house_dls = soup.find_all("dl", class_="list hiddenMap rel")
            
            # 提取并过滤当前页有效租房数据
            current_page_data = []
            for idx, house_dl in enumerate(house_dls, 1):
                rent_info = extract_rent_info(house_dl)
                # 过滤无效数据：面积和月租金至少有一个不为空
                if rent_info["面积"] or rent_info["月租金"]:
                    current_page_data.append(rent_info)
                    # 打印日志，确认数据已提取
                    print(f"  已提取第{idx}条租房房源：面积={rent_info['面积']} | 月租金={rent_info['月租金']}")
            
            if current_page_data:
                print(f"✅ 租房第{page}页第{retry_count}次重试成功，获取{len(current_page_data)}条数据")
                return current_page_data
            else:
                print(f"⚠️ 租房第{page}页第{retry_count}次重试无有效数据，继续...")
                time.sleep(RETRY_DELAY)  # 重试间隔

        except TimeoutException:
            print(f"⚠️ 租房第{page}页第{retry_count}次重试：页面加载超时，继续...")
            time.sleep(RETRY_DELAY)
        except Exception as e:
            print(f"⚠️ 租房第{page}页第{retry_count}次重试：错误 {str(e)[:60]}，继续...")
            time.sleep(RETRY_DELAY)
    
    # 超过最大重试次数，返回空列表
    print(f"❌ 租房第{page}页重试超过{MAX_RETRIES}次，无有效房源")
    return []


# --------------------------
# 5. 多页爬取主逻辑
# --------------------------
def crawl_multi_page(driver):
    all_rent_data = []
    print(f"\n{'='*60}")
    print(f"开始爬取租房信息，最多{TARGET_PAGES}页（房源为空则停止）")
    print(f"{'='*60}")

    for page in range(1, TARGET_PAGES + 1):
        print(f"\n📄 处理租房第{page}/{TARGET_PAGES}页")
        
        # 爬取当前页（带重试机制）
        current_page_data = crawl_single_page(driver, page)
        
        if not current_page_data:
            print(f"⚠️ 租房第{page}页无有效房源，停止翻页")
            break  # 停止后续页面爬取
        
        # 标记数据来源页，添加到总列表
        for data in current_page_data:
            data["来源页"] = f"第{page}页"
        all_rent_data.extend(current_page_data)

    df = pd.DataFrame(all_rent_data)
    print(f"\n{'='*60}")
    print(f"租房信息爬取完成，共{len(df)}条数据")
    print(f"{'='*60}")
    return df


# --------------------------
# 6. 保存租房数据到Excel
# --------------------------
def save_to_excel(rent_df):
    """将租房数据保存到macOS的「下载」文件夹"""
    user_home = os.path.expanduser("~")
    excel_path = os.path.join(user_home, "Downloads", "hebei_zu_xiahuyuan_data.xlsx")
    
    try:
        with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
            rent_df.to_excel(writer, sheet_name="租房信息", index=False)
        print(f"\n📁 租房数据保存成功！Excel文件路径：")
        print(f"   {excel_path}")
    except Exception as e:
        print(f"\n❌ 保存Excel失败：{str(e)}")


# --------------------------
# 7. 主执行流程
# --------------------------
if __name__ == "__main__":
    # 初始化浏览器
    driver = init_browser()
    try:
        # 执行多页爬取租房信息
        rent_data = crawl_multi_page(driver)
        # 保存租房数据到Excel
        save_to_excel(rent_data)
    finally:
        # 关闭浏览器释放资源
        driver.quit()
        print(f"\n{'='*60}")
        print("Chrome浏览器已关闭，爬取流程结束")
        print(f"{'='*60}")


开始爬取租房信息，最多5页（房源为空则停止）

📄 处理租房第1/5页
[第1次重试] 正在加载租房第1页：https://zhangjiakou.zu.fang.com/house-a014963/
  已提取第1条租房房源：面积=77㎡ | 月租金=1000元/月
  已提取第2条租房房源：面积=30㎡ | 月租金=800元/月
  已提取第3条租房房源：面积=86㎡ | 月租金=800元/月
  已提取第4条租房房源：面积=32㎡ | 月租金=700元/月
  已提取第5条租房房源：面积=85㎡ | 月租金=900元/月
  已提取第6条租房房源：面积=85㎡ | 月租金=800元/月
  已提取第7条租房房源：面积=35㎡ | 月租金=700元/月
  已提取第8条租房房源：面积=32㎡ | 月租金=700元/月
  已提取第9条租房房源：面积=32㎡ | 月租金=700元/月
  已提取第10条租房房源：面积=32㎡ | 月租金=1500元/月
  已提取第11条租房房源：面积=32㎡ | 月租金=700元/月
  已提取第12条租房房源：面积=32㎡ | 月租金=700元/月
  已提取第13条租房房源：面积=60㎡ | 月租金=700元/月
  已提取第14条租房房源：面积=65㎡ | 月租金=600元/月
  已提取第15条租房房源：面积=64㎡ | 月租金=650元/月
  已提取第16条租房房源：面积=34㎡ | 月租金=800元/月
  已提取第17条租房房源：面积=29㎡ | 月租金=700元/月
  已提取第18条租房房源：面积=29㎡ | 月租金=850元/月
  已提取第19条租房房源：面积=85㎡ | 月租金=1100元/月
  已提取第20条租房房源：面积=85㎡ | 月租金=1000元/月
  已提取第21条租房房源：面积=85㎡ | 月租金=1200元/月
  已提取第22条租房房源：面积=38㎡ | 月租金=500元/月
  已提取第23条租房房源：面积=30㎡ | 月租金=600元/月
  已提取第24条租房房源：面积=77㎡ | 月租金=900元/月
  已提取第25条租房房源：面积=56㎡ | 月租金=700元/月
  已提取第26条租房房源：面积=125㎡ | 月租金=1100元/月
  已提取第27条租房房源：面积